The goal of this notebook is to use Boruta_Py to assess feature importance in more complex models

In [1]:
import shap

In [8]:
import pickle
from boruta import BorutaPy
import pandas as pd
from predict_protein import download_cptac, select_features, train_model, utils
import matplotlib.pyplot as plt
import numpy as np

model_loc = '/Volumes/Ginkgo_Data/Data/2021-11_CPTACTrainedModels/out3'

In [3]:
cptac_list = download_cptac.download_cptac(n_tumors=8)
tumor_df = pd.concat(cptac_list)
tm = train_model.LearnCPTAC(tumor_df)


cptac warning: transcriptomics data was not found for the following samples, so transcriptomics data columns were filled with NaN for these samples: C3L-00563.N, C3L-00605.N, C3L-00769.N, C3L-00770.N, C3L-00771.N, C3L-00930.N, C3L-00947.N, C3L-00963.N, C3L-01246.N, C3L-01249.N, C3L-01252.N, C3L-01256.N, C3L-01257.N, C3L-01744.N, C3N-00200.N, C3N-00729.N, C3N-01211.N, NX1.N, NX10.N, NX11.N, NX12.N, NX13.N, NX14.N, NX15.N, NX16.N, NX17.N, NX18.N, NX2.N, NX3.N, NX4.N, NX5.N, NX6.N, NX7.N, NX8.N, NX9.N (/Users/edwardlau/PythonProjects/CPTAC_Protein/predict_protein/download_cptac.py, line 38)
cptac warning: proteomics data was not found for the following samples, so proteomics data columns were filled with NaN for these samples: 01OV002, 01OV008, 01OV010, 01OV013, 01OV019, 01OV045, 01OV046, 01OV049, 02OV035, 02OV040, 02OV042, 02OV045, 04OV041, 11OV009, 13OV004, 17OV019, 17OV034, 22OV001, 26OV010 (/Users/edwardlau/PythonProjects/CPTAC_Protein/predict_protein/download_cptac.py, line 42)
cptac

In [4]:
def get_model(tm_obj, model_loc, model, protein_todo):

    with open(model_loc, 'rb') as f:
        model_results = pickle.load(f)

    protein_model  = [result['model'] for result in model_results if result['metrics'].index[0] == protein_todo][0]

    # Get features and then do train test split again
    tm_obj.included_features = model
    print(tm_obj.included_features)
    x_train, x_test, y_train, y_test = tm_obj.get_train_test(protein_to_do=protein_todo)

    # Output a pandas data frame with y (actual mass spec protein level) and y_hat (predicted from transcript(s))
    train_df = pd.DataFrame({'y': y_train, 'yhat': protein_model.predict(x_train[protein_model.feature_names_in_]), 'set': 'train'})
    test_df = pd.DataFrame({'y': y_test, 'yhat': protein_model.predict(x_test[protein_model.feature_names_in_]), 'set': 'test'})

    return protein_model, x_test, y_test # pd.concat([train_df, test_df])


In [35]:
for model in ['corum', ]: # 'single', 'corum', 'stringhi', 'string'
    for protein in ['CMC1', 'PCCB', 'PSMG2', 'SMCR8']:
        m, X, y = get_model(tm_obj=tm,
                             model_loc=f'{model_loc}/tumor8_forest_{model}.p',
                             model=model,
                             protein_todo=protein,
                        )
        # Reorder the X data frame
        X = X[m.feature_names_in_]
        #%%
        feat_selector = BorutaPy(m, n_estimators='auto', verbose=1, random_state=1)
        feat_selector.fit(X.values, y.values.ravel())

        #%%
        print(X.columns[feat_selector.support_].to_list())




corum
Iteration: 	1 / 100
Confirmed: 	0
Tentative: 	13
Rejected: 	0
Iteration: 	2 / 100
Confirmed: 	0
Tentative: 	13
Rejected: 	0
Iteration: 	3 / 100
Confirmed: 	0
Tentative: 	13
Rejected: 	0
Iteration: 	4 / 100
Confirmed: 	0
Tentative: 	13
Rejected: 	0
Iteration: 	5 / 100
Confirmed: 	0
Tentative: 	13
Rejected: 	0
Iteration: 	6 / 100
Confirmed: 	0
Tentative: 	13
Rejected: 	0
Iteration: 	7 / 100
Confirmed: 	0
Tentative: 	13
Rejected: 	0
Iteration: 	8 / 100
Confirmed: 	4
Tentative: 	6
Rejected: 	3
Iteration: 	9 / 100
Confirmed: 	4
Tentative: 	6
Rejected: 	3
Iteration: 	10 / 100
Confirmed: 	4
Tentative: 	6
Rejected: 	3
Iteration: 	11 / 100
Confirmed: 	4
Tentative: 	6
Rejected: 	3
Iteration: 	12 / 100
Confirmed: 	4
Tentative: 	6
Rejected: 	3
Iteration: 	13 / 100
Confirmed: 	4
Tentative: 	6
Rejected: 	3
Iteration: 	14 / 100
Confirmed: 	4
Tentative: 	6
Rejected: 	3
Iteration: 	15 / 100
Confirmed: 	4
Tentative: 	6
Rejected: 	3
Iteration: 	16 / 100
Confirmed: 	5
Tentative: 	5
Rejected: 	3
Iter

array([4, 1, 4, 1, 1, 6, 7, 1, 1, 1, 2, 3, 7])